# ETL Pipeline Preparation
### 1. Import libraries and load datasets.
- Import Python libraries


In [260]:
# import libraries
import sqlite3
import pandas as pd
import numpy as np
from sqlalchemy import create_engine 

In [210]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [212]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merging the messages and categories datasets using the common id
- Assigning this combined dataset to `df`, which will be cleaned in the following steps

In [213]:
# merge datasets
df = messages.merge(categories, how='outer', on=['id'])
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Splitting the values in the `categories` column on the `;` character so that each value becomes a separate column. 
- Using the first row of categories dataframe to create column names for the categories data.
- Renaming columns of `categories` with new column names.

In [214]:
# create a dataframe of the 36 individual category columns
categories = df['categories'].str.split(';', expand=True, n=36)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [215]:
# select the first row of the categories dataframe
row = categories[1:2]
row

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [216]:
# using this row to extract a list of new column names for categories.
category_colnames = row.apply(lambda x : x.str.slice(0,-2))
print(category_colnames)

        0        1      2            3             4                 5   \
1  related  request  offer  aid_related  medical_help  medical_products   

                  6         7         8            9   ...           26  \
1  search_and_rescue  security  military  child_alone  ...  aid_centers   

                     27               28      29     30    31          32  \
1  other_infrastructure  weather_related  floods  storm  fire  earthquake   

     33             34             35  
1  cold  other_weather  direct_report  

[1 rows x 36 columns]


In [217]:
category_colnames_list = category_colnames.values.tolist()

In [218]:
# rename the columns of `categories`
categories.columns = category_colnames_list
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterating through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Converting the string to a numeric value.

In [222]:
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [223]:
categories_clean = categories.copy()

In [224]:
for column in category_colnames_list:
    # set each value to be the last character of the string
    categories_clean[column] = categories_clean[column].apply(lambda x : x.str.slice(-1))
    
    # convert column from string to numeric
    categories_clean[column] = categories_clean[column].astype(int)


In [275]:
categories_clean.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [277]:
# Confirm values in each column of the categories dataframe are, in fact, only 0's and 1's.
for column in categories_clean:
    if len(np.unique(categories_clean[column])) != 2:
        print(column, np.unique(categories_clean[column]))

related [0 1 2]
child_alone [0]


In [276]:
# Count number of entries where related == 2
categories_clean['related'].value_counts()

1    20042
0     6140
2      204
Name: related, dtype: int64

In [279]:
# Drop child_alone from categories dataframe.
categories_clean.drop('child_alone', axis = 1, inplace = True)

categories_clean.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Droping the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [226]:
# drop the original categories column from `df`
df_clean_one = df.drop(columns=['categories'])

df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [227]:
df_clean_one.columns

Index(['id', 'message', 'original', 'genre'], dtype='object')

In [282]:
# concatenate the original dataframe with the new `categories` dataframe
df_clean_two = pd.concat([df_clean_one, categories_clean],join='inner', axis=1)
df_clean_two.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


### 6. Remove duplicates.
- Checking how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [283]:
# check number of duplicates
print('There are', sum(df_clean_two.duplicated()), 'duplicates in the dataset.')

There are 170 duplicates in the dataset.


In [288]:
# drop duplicates
df_clean_two.drop_duplicates(inplace = True)

In [289]:
# check number of duplicates
print('There are', sum(df_clean_two.duplicated()), 'duplicates in the dataset.')

There are 0 duplicates in the dataset.


In [290]:
# Get messages with a related value of 2
df_clean_two['message'][df_clean_two['related'] == 2]

117      Dans la zone de Saint Etienne la route de Jacm...
221      . .. i with limited means. Certain patients co...
307      The internet caf Net@le that's by the Dal road...
462      Bonsoir, on est a bon repos aprs la compagnie ...
578      URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX ...
657      elle est vraiment malade et a besoin d'aide. u...
889      no authority has passed by to see us. We don't...
903      It's Over in Gressier. The population in the a...
931      we sleep with the baby. Thanks in advance for ...
937      I need help in Jrmie because I was in Port-au-...
939      fsa pou mwen s v p map mouri mwen gen tout po ...
1234     .. Gonaives, in a place called Canal Bois in F...
1256     GEN YON TIBEBE KI MALAD NAN KOU PASKE BLOK TON...
1317     don't understand the first part. .. understand...
1409     Est-ce que ya monde qui aller U. s. a et qui o...
1506     Gens ont information qui dit que si on moins c...
1694         This is my address : Cersine 8 Prolong. .. 

In [292]:
# Remove rows with a related value of 2 from the dataset
df_clean_two = df_clean_two[df_clean_two['related'] != 2]

### 7. Save the clean dataset into an sqlite database.


In [294]:
engine = create_engine('sqlite:///disaster-data.db')
df_clean_two.to_sql('messages', engine, index=False)

### 8. Using this notebook to complete `etl_pipeline.py`

### 9. Testing the database

In [295]:
conn = sqlite3.connect('disaster-data.db')

In [296]:
medical_help = pd.read_sql('SELECT "message", "genre" FROM messages WHERE medical_help == 1', conn)



In [297]:
medical_help.message[0]

'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )'

In [298]:
df_clean_two.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [322]:
df_y_data = df_clean_two.iloc[:,4:]

In [323]:
df_y_data.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [328]:
columns_names = list(df_y_data.columns)

In [342]:
sum_of_data = df_y_data.sum().sort_values(ascending=False)

In [353]:
x = list(sum_of_data.index)

In [358]:
y = list(sum_of_data.values)

In [351]:
sum_of_data.sort_values(ascending=False)

related                   19906
aid_related               10860
weather_related            7297
direct_report              5075
request                    4474
other_aid                  3446
food                       2923
earthquake                 2455
storm                      2443
shelter                    2314
floods                     2155
medical_help               2084
infrastructure_related     1705
water                      1672
other_weather              1376
buildings                  1333
medical_products           1313
transport                  1201
death                      1194
other_infrastructure       1151
refugees                    875
military                    860
search_and_rescue           724
money                       604
electricity                 532
cold                        530
security                    471
clothing                    405
aid_centers                 309
missing_people              298
hospitals                   283
fire    

In [347]:
list(sum_of_data.index)

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [337]:
sum_of_data = pd.DataFrame(sum_of_data)

In [341]:
sum_of_data

,0
related,19906
request,4474
offer,118
aid_related,10860
medical_help,2084
medical_products,1313
search_and_rescue,724
security,471
military,860
water,1672


In [325]:
df_y_data['aid_related'].sum()

10860

In [300]:
df_clean_two['related'].value_counts()

1    19906
0     6122
Name: related, dtype: int64

In [301]:
related = df_clean_two[df_clean_two['related'] == 0]

In [317]:
related['message'][:20]

5                 Information about the National Palace-
8        I would like to receive the messages, thank you
11     I am in Petionville. I need more information r...
17     are you going to call me or do you want me to ...
18        I don't understand how to use this thing 4636.
25                    Can you tell me about this service
30     Good evening, Radio one please. I would like i...
32     I'm here, I didn't find the person that I need...
52     I'm listening to you at Miraguan we asking the...
66           i am very happy, i hear god, religious hyme
79         I would like to know how food is distributed.
80     I can play carnaval Anbasad Camp there I think...
82     ples give us cell cards ( cell phone minuts ) ...
83     Whoever sees her in the streets : she was wear...
87         good people help us. god bless you and us too
96          Digicel let me go through, it's an emergency
97                 Where can I find Capital Bank please?
103    URGENT ACTION NEEDED --t

In [316]:
df_clean_two['message'][:20]

0     Weather update - a cold front from Cuba that c...
1               Is the Hurricane over or is it not over
2                       Looking for someone but no name
3     UN reports Leogane 80-90 destroyed. Only Hospi...
4     says: west side of Haiti, rest of the country ...
5                Information about the National Palace-
6                        Storm at sacred heart of jesus
7     Please, we need tents and water. We are in Sil...
8       I would like to receive the messages, thank you
9     I am in Croix-des-Bouquets. We have health iss...
10    There's nothing to eat and water, we starving ...
11    I am in Petionville. I need more information r...
12    I am in Thomassin number 32, in the area named...
13    Let's do it together, need food in Delma 75, i...
14    More information on the 4636 number in order f...
15    A Comitee in Delmas 19, Rue ( street ) Janvier...
16    We need food and water in Klecin 12. We are dy...
17    are you going to call me or do you want me

In [318]:
len(df_clean_two)

26028